###*Load dataset* 

In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()


57344/57026 [==============================] - 0s 0us/step


In [ ]:
train_data.shape

(404, 13)

In [ ]:
test_data.shape

(102, 13)

In [ ]:
train_data[7]

array([  4.0974,   0.    ,  19.58  ,   0.    ,   0.871 ,   5.468 ,
       100.    ,   1.4118,   5.    , 403.    ,  14.7   , 396.9   ,
        26.42  ])

In [ ]:
train_targets[0]

15.2

###  Preparing the data

#### Normalizition

In [9]:
std = train_data.std(axis=0)
mean = train_data.mean(axis=0)
train_data -= mean



train_data /= std


In [10]:
std = train_data.std(axis=0)

train_data /= std
test_data /= std
test_data -= mean

In [11]:
mean

array([ 1.92365376e-17,  6.48546123e-17, -5.11142284e-17,  3.18776908e-17,
       -8.24423038e-18,  7.14499966e-18, -1.81373068e-17,  1.86869222e-17,
        1.95113452e-17, -3.43509599e-17, -7.69461502e-17, -5.57859589e-17,
       -1.70380761e-17])

In [12]:
std

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
train_data[7]

array([ 0.0381708 , -0.48361547,  1.24588095, -0.25683275,  2.67733525,
       -1.12719983,  1.11048828, -1.14833073, -0.51114231, -0.01744323,
       -1.71818909,  0.44807713,  1.88793986])

In [14]:
np.array(train_data.mean(axis=0),dtype=np.int8)

NameError: ignored

In [15]:
train_data.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [16]:
import numpy as np
np.mean(train_data, axis=0)

array([ 1.48396147e-17, -2.19846143e-17, -4.78165362e-17, -8.27171115e-17,
       -8.24423038e-18,  1.56640377e-17, -1.81373068e-17,  1.09923072e-18,
        1.07174995e-17, -3.21524985e-17,  6.70530738e-17, -6.27248528e-17,
       -8.24423038e-18])

#### Building your network

In [17]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
model = build_model()
history = model.fit(partial_train_data, partial_train_targets,
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=1, verbose=0)
mae_history = history.history['val_mae']
all_mae_histories.append(mae_history)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [ ]:
history.history.keys()

#### Building 

In [ ]:
average_mae_history = [
np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
average_mae_history

#### Training the final model

In [ ]:
model = build_model()
model.fit(train_data, train_targets,
          epochs=100, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mse_score

In [ ]:
test_mae_score

You’re still off by about $2,550.

In [ ]:
model.predict(test_data)